In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import NearestNeighbors

# Load the dataset
file_path = '25000_movie_subset.csv'  # Replace with your dataset file path
movies = pd.read_csv(file_path)

# Features to use
numerical_features = ["popularity", "vote_average", "runtime"]
categorical_features = ["genres", "original_language"]

# Scale numerical features
scaler = MinMaxScaler()
numerical_scaled = scaler.fit_transform(movies[numerical_features])

# One-hot encode categorical features
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
categorical_encoded = onehot_encoder.fit_transform(movies[categorical_features])

# Combine numerical and categorical features
features_combined = pd.concat(
    [pd.DataFrame(numerical_scaled), pd.DataFrame(categorical_encoded)],
    axis=1
).values

# Train the KNN model
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(features_combined)

# Function to recommend movies
def recommend_movies_knn(movie_title, n_recommendations=5):
    try:
        # Find the index of the movie
        movie_idx = movies[movies['title'].str.lower() == movie_title.lower()].index[0]
        
        # Get the feature vector for the movie
        movie_vector = features_combined[movie_idx].reshape(1, -1)
        
        # Find the nearest neighbors
        distances, indices = knn.kneighbors(movie_vector, n_neighbors=n_recommendations + 1)
        
        # Get the titles of the recommended movies
        recommended_movies = movies.iloc[indices[0][1:]]['title'].tolist()
        return recommended_movies
    except IndexError:
        return f"Movie '{movie_title}' not found in the dataset."

# Example usage
movie_to_search = "Don 2"  # Replace with a valid movie title
recommended_movies = recommend_movies_knn(movie_to_search)

# Output recommendations
print(f"Movies similar to '{movie_to_search}':")
print(recommended_movies)


Movies similar to 'Don 2':
['Karz', 'Mardangi', 'Raksha', 'Sky Force', 'Dada Lakhmi']
